In [1]:
import ROOT as r
import json

from pathlib import Path

Welcome to JupyROOT 6.24/06


In [2]:
dataset_runnable = json.loads(Path("af_v2_onefile.json").read_text())
file_to_open=list(dataset_runnable['Znunugamma']['files'].keys())[0]
f=r.TFile(file_to_open,"RO")
t=f.Get("analysis")

h_baseline_pt=r.TH1F("baseline_pt","baseline_pt",10000,0,10000)

h_istruth_it_ii_pt=r.TH1F("istruth_istight_isiso_pt","istight_isiso",10000,0,10000)
h_istruth_it_ni_pt=r.TH1F("istruth_istight_noiso_pt","istight_noiso",10000,0,10000)
h_istruth_nt_ii_pt=r.TH1F("istruth_notight_isiso_pt","notight_isiso",10000,0,10000)
h_istruth_nt_ni_pt=r.TH1F("istruth_notight_noiso_pt","notight_noiso",10000,0,10000)

h_notruth_it_ii_pt=r.TH1F("notruth_istight_isiso_pt","istight_isiso",10000,0,10000)
h_notruth_it_ni_pt=r.TH1F("notruth_istight_noiso_pt","istight_noiso",10000,0,10000)
h_notruth_nt_ii_pt=r.TH1F("notruth_notight_isiso_pt","notight_isiso",10000,0,10000)
h_notruth_nt_ni_pt=r.TH1F("notruth_notight_noiso_pt","notight_noiso",10000,0,10000)

eventcount=0
premindphicut=0
prejetscuts=0
postleadjetptcut=0
postmindphicut=0
postphotonpresel=0
totalevents=t.GetEntriesFast()
for e in t:

    eventcount+=1
    if eventcount%50000 == 0:
        print(f"Processed {eventcount:6d}/{totalevents} events")
    #if eventcount>10000:
    #    break
    
    #=================================================================
    # preselection
    #
    met=e.met_met_NOSYS
    met_phi=e.met_phi_NOSYS
    met_tlv=r.TLorentzVector()
    met_tlv.SetPtEtaPhiM(met,0,met_phi,0)

    # MET requirement, for now just look at low MET
    if met>=250000.: continue

    # veto events with leptons
    nelectrons = len(e.el_pt_NOSYS)
    nmuons     = len(e.mu_pt_NOSYS)
    if (nelectrons+nmuons)>0: continue

    ## overlap removal for vgamma samples
    if not e.in_vgamma_overlap_7: 
        continue

    prejetscuts+=1

    # no b-tagged jets, at least one jet with pT>100 GeV, no jets near MET
    jets=[]
    mindphijetsmet=999.
    btags=0
    leadjetpt=0.
    ljpt=max(e.jet_pt_NOSYS)
    if e.jet_pt_NOSYS[0]==None:
        print(e.jet_pt_NOSYS)
    for j in range(len(e.jet_pt_NOSYS)):
        tlv=r.TLorentzVector()
        tlv.SetPtEtaPhiM(e.jet_pt_NOSYS[j],
                         e.jet_eta[j],
                         e.jet_phi[j],
                         0)
        jets.append(tlv)
        mindphijetsmet=min(mindphijetsmet,abs(tlv.DeltaPhi(met_tlv)))
        if ord(e.jet_btag_select[j])>0: btags+=1
        #if int.from_bytes(bytes(e.jet_btag_select[j],'utf8'),'big')>0: btags+=1
        leadjetpt=max(leadjetpt,e.jet_pt_NOSYS[j])

    if ljpt!=leadjetpt:
        print(e.jet_pt_NOSYS)
    if leadjetpt<100000.:
        continue

    postleadjetptcut+=1
    if btags>0: continue

    premindphicut+=1
    
    if mindphijetsmet<0.4: continue

    postmindphicut+=1

    # at least one loose photon with pT>10 GeV, no photons near jets
    photons={}
    photon_index=-1
    for p in range(len(e.ph_pt_NOSYS)):
        if not ord(e.ph_select_baseline_NOSYS[p])>0: 
            continue

        tlv=r.TLorentzVector()
        tlv.SetPtEtaPhiM(e.ph_pt_NOSYS[p],
                         e.ph_eta[p],
                         e.ph_phi[p],
                         0)

        if tlv.Pt()<10000.:
            continue
        
        if (abs(tlv.Eta())>2.37) or (abs(tlv.Eta())>1.37 and abs(tlv.Eta())<1.52):
            continue

        # not clear we need this, should be done via overlap removal?  not applying for now.
        #mindrphjet=999
        #for j in jets:
        #    mindrphjet=min(mindrphjet,tlv.DeltaPhi(j))
        #if mindrphjet<0.4: continue
        if not ord(e.ph_select_or_dR02Ph_NOSYS[p]) > 0:
            continue

        if ((e.ph_isEM_NOSYS[p] & 0x45fc01) != 0):
            continue
        
        if photon_index<0:
            photon_index=p


    if photon_index<0:
        continue

    postphotonpresel+=1
    
    h_baseline_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)

    #=================================================================

    #=================================================================
    # now we have the index of the photon to keep.  Now figure out if this is a truth-matched photon or not.
    # will need to fix this whenever Giordon et al decide on a truth matching scheme.
    #print(f"{e.ph_truthType[photon_index]} {e.ph_truthOrigin[photon_index]} {e.ph_truthpdgId[photon_index]}")
    ph_truthmatch=((e.ph_truthType[photon_index]!=16) and (e.ph_truthType[photon_index]!=0))

    # the tight and isolated flags are easier:
    ph_tight=(ord(e.ph_select_tightID_NOSYS [photon_index])>0)
    ph_iso  =(ord(e.ph_select_tightIso_NOSYS[photon_index])>0)

    if ph_truthmatch:
        if       ph_tight and     ph_iso: h_istruth_it_ii_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
        elif     ph_tight and not ph_iso: h_istruth_it_ni_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
        elif not ph_tight and     ph_iso: h_istruth_nt_ii_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
        elif not ph_tight and not ph_iso: h_istruth_nt_ni_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
    else:
        if       ph_tight and     ph_iso: h_notruth_it_ii_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
        elif     ph_tight and not ph_iso: h_notruth_it_ni_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
        elif not ph_tight and     ph_iso: h_notruth_nt_ii_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
        elif not ph_tight and not ph_iso: h_notruth_nt_ni_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
    #=================================================================

Processed  50000/872532 events
Processed 100000/872532 events
Processed 150000/872532 events
Processed 200000/872532 events
Processed 250000/872532 events
Processed 300000/872532 events
Processed 350000/872532 events
Processed 400000/872532 events
Processed 450000/872532 events
Processed 500000/872532 events
Processed 550000/872532 events
Processed 600000/872532 events
Processed 650000/872532 events
Processed 700000/872532 events
Processed 750000/872532 events
Processed 800000/872532 events
Processed 850000/872532 events


Error in <TList::Clear>: A list is accessing an object (0x7573a40) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x759ffe0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x75a0410) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x75a9220) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x75a9590) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x75a9a00) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x75a9ea0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x75aa270) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x75aa740) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x75abaa0) alread

In [3]:
print(f"pre jets cuts {prejetscuts}") 
print(f"post leadjet  {postleadjetptcut}")
print(f"pre mindphi   {premindphicut}")  
print(f"post mindphi  {postmindphicut}")
print(f"post photonpr {postphotonpresel}")
print(f"                 baseline: {h_baseline_pt.GetSumOfWeights()}")

print(f" truth,  tight,  isolated: {h_istruth_it_ii_pt.GetSumOfWeights()}")
print(f" truth,  tight, !isolated: {h_istruth_it_ni_pt.GetSumOfWeights()}")
print(f" truth, !tight,  isolated: {h_istruth_nt_ii_pt.GetSumOfWeights()}")
print(f" truth, !tight, !isolated: {h_istruth_nt_ni_pt.GetSumOfWeights()}")

print(f"!truth,  tight,  isolated: {h_notruth_it_ii_pt.GetSumOfWeights()}")
print(f"!truth,  tight, !isolated: {h_notruth_it_ni_pt.GetSumOfWeights()}")
print(f"!truth, !tight,  isolated: {h_notruth_nt_ii_pt.GetSumOfWeights()}")
print(f"!truth, !tight, !isolated: {h_notruth_nt_ni_pt.GetSumOfWeights()}")

pre jets cuts 266345
post leadjet  260399
pre mindphi   242951
post mindphi  189045
post photonpr 154545
                 baseline: 154545.0
 truth,  tight,  isolated: 99704.0
 truth,  tight, !isolated: 41973.0
 truth, !tight,  isolated: 5727.0
 truth, !tight, !isolated: 3970.0
!truth,  tight,  isolated: 330.0
!truth,  tight, !isolated: 1023.0
!truth, !tight,  isolated: 351.0
!truth, !tight, !isolated: 1467.0
